In [226]:
from unidecode import unidecode
#print(unidecode("ąąąBZWDAWD").lower())
def load_data(file):
    ans = []
    with open(file) as f:
        for i in range(10**6):
            line = f.readline()
            ans += line.split()
    return ans
unigrams = {}

In [227]:
def convert_to_alnum(w):
    return ''.join(filter(str.isalnum, w))
def generate_1grams(data):
    for word in data:
        word = convert_to_alnum(word)
        norm_word = convert_to_alnum( unidecode(word).lower() )
        if norm_word != '':
            if norm_word not in unigrams:
                unigrams[norm_word] = {}
            if word not in unigrams[norm_word]:
                unigrams[norm_word][word] = 0
            unigrams[norm_word][word] += 1

def load_Ngrams(file):
    K = 5
    bigrams = {}
    with open(file, 'r') as f:
        for line in f:
            count = int(line.split()[0])
            w1 = convert_to_alnum(line.split()[1])
            w2 = convert_to_alnum(line.split()[2])
            if count >= K:
                
                if w1 not in bigrams:
                    bigrams[w1] = {}
                bigrams[w1][w2] = count
                
                norm_word1 = convert_to_alnum(unidecode(w1).lower() )
                norm_word2 = convert_to_alnum(unidecode(w2).lower() )
                if norm_word1 != '':
                    if norm_word1 not in unigrams:
                        unigrams[norm_word1] = {}
                    if w1 not in unigrams[norm_word1]:
                        unigrams[norm_word1][w1] = 0
                    unigrams[norm_word1][w1] += 1
                if norm_word2 != '':
                    if norm_word2 not in unigrams:
                        unigrams[norm_word2] = {}
                    if w2 not in unigrams[norm_word2]:
                        unigrams[norm_word2][w2] = 0
                    unigrams[norm_word2][w2] += 1
    return bigrams


In [439]:
def generate2grams(data):
    bigrams_u = {}
    key = data[0]
    key = convert_to_alnum(key)
    norm_key = convert_to_alnum( unidecode(key).lower() )
    for word in data[1:]:
        word = convert_to_alnum(word)
        norm_word = convert_to_alnum( unidecode(word).lower() )
        if norm_key not in bigrams_u:
            bigrams_u[norm_key] = {}
        if norm_word not in bigrams_u[norm_key]:
            bigrams_u[norm_key][norm_word] = {}
        if (key,word) not in bigrams_u[norm_key][norm_word]:
            bigrams_u[norm_key][norm_word][(key,word)] = 0
        bigrams_u[norm_key][norm_word][(key,word)] += 1
        key = word
        norm_key = norm_word
    return bigrams_u
            
        

In [228]:
def load_tests(file):
    ans = []
    with open(file) as f:
        for i in range(10**6 + 2*10**5):
            line = f.readline()
            if i >= 10**6:
                ans.append(line.split())
    return ans

In [269]:
import re
good_names = {}
def load_polimorfologik(file):
    with open(file) as f:
        for i in range(10**6):
            line = f.readline()
            if line[0].isupper():
                word = re.split('\;',line)[1]
                norm_word = convert_to_alnum( unidecode(word).lower() )
                if norm_word not in good_names:
                    good_names[norm_word] = {}
                good_names[norm_word][word] = 1

In [270]:
load_polimorfologik("polimorfologik-2.1.txt")

In [229]:
tests = load_tests("polish_corpora.txt")

In [230]:
data = load_data("polish_corpora.txt")

In [233]:
generate_1grams(data)

In [440]:
bigrams_u = generate2grams(data)

In [234]:
bigrams = load_Ngrams('poleval_2grams.txt')

In [442]:
import random
import numpy as np

def norm_sen(s):
    ans = []
    for w in s:
        tmp = convert_to_alnum(unidecode(w).lower())
        if len(tmp) > 0:
            ans.append(tmp)
    return ans

def find_word(w,bef = False,nxt = False):
    #print(w, bef, nxt)
    ws = list(unigrams[w].keys())
    cs = list(unigrams[w].values())
    words_tmp = [(cs[i],ws[i]) for i in range(len(ws))]
    words_tmp.sort(reverse = True)
    words = [words_tmp[i][1] for i in range(len(ws))]
    if words[0][0].isupper():
        return words[0]
    if len(words) > 1 and words[1][0].isupper() and words_tmp[0][0] - words_tmp[1][0] < 10:
        #if len(words[1]) > 1 and not words[1][1].isupper():
        return words[1]
    if all(words_tmp[0][0] >= 9*i[0] for i in words_tmp if i != words_tmp[0]):
        return words[0]
    best = []
    best_nxt = []
    best_bef = []
    bef_s = set()
    nxt_s = set()
    upper_cases_b = []
    upper_cases_a = []
    if nxt:
        nxt_b = find_word(nxt)
    for e in words:
        t = convert_to_alnum( e.lower() )
        tb = unidecode(bef.lower())
        if bef:
            if bef in bigrams and t in bigrams[bef] and bigrams[bef][t] not in bef_s:
                best_bef.append((111*bigrams[bef][t], e))
                bef_s.add(bigrams[bef][t])
            if  tb in bigrams_u and t in bigrams_u[tb] and (bef,e) in bigrams_u[tb][t]:
                upper_cases_b.append(bigrams_u[tb][t][(tb,t)], e)
                
        if nxt:
            if t in bigrams and nxt_b in bigrams[t] and bigrams[t][nxt_b] not in nxt_s:
                best_nxt.append((bigrams[t][nxt_b], e))
                nxt_s.add(bigrams[t][nxt_b])
        
            if t in bigrams_u and nxt in bigrams_u[t] and (e,nxt_b) in bigrams_u[t][nxt]:
                upper_cases_a.append(bigrams_u[t][nxt][(e,nxt_b)], e)
                
    upper_cases_b.sort(reverse = True)
    upper_cases_a.sort(reverse = True)
    best_bef.sort(reverse = True)
    best_nxt.sort(reverse = True)
    #if w == 'dbac':
        #print(best_bef, best_nxt)
        #print(words)
    if len(best_bef) > 1 and best_bef[0][0] > 10*best_bef[1][0]:
        best = best_bef
    else:
        if len(best_nxt) > 1 and len(best_bef) > 1 and best_nxt[0][0] > 10*best_nxt[1][0]:
            best = best_nxt
        else:
            best = best_bef + best_nxt
    best.sort(reverse = True)
    #r = np.random.choice(len(words), p = [p_r/sum_probs for p_r in probs])
    if len(best) > 0:
        if best[0][1][0].upper() + best[0][1][1:] == 
        return best[0][1]
    return words[0]

def sen_restoration(s):
    ans = []
    for i in range(len(s)):
        #print(s[i])
        bef = False
        nxt = False
        if i < len(s) - 1:
            nxt = s[i + 1].lower()
        if i > 0:
            bef = ans[i - 1].lower()
        if nxt not in unigrams:
            nxt = False
        if s[i] not in unigrams:
            new_w = s[i]
        else:
            new_w = find_word(s[i],bef,nxt)
        if len(ans) == 0:
            new_w = new_w[0].upper() + new_w[1:]
        ans.append(new_w)
    return ans
        

In [430]:
import math
def good_sen(s):
    ans = []
    for w in s:
        tmp = convert_to_alnum(w)
        if len(tmp) > 0:
            ans.append(tmp)
    return ans

def score(good_sen, restorated_sen):
    g1 = 0
    g2 = 0
    for i in range(len(good_sen)):
        if good_sen[i].lower() == restorated_sen[i].lower():
            g1 += 1
        if good_sen[i] == restorated_sen[i]:
            g2 += 1
        else:
            print(restorated_sen)
            print(good_sen)
            #print(restorated_sen[i],good_sen[i])
        
    a = g1 / len(good_sen)
    b = g2 / len(good_sen)
    return math.sqrt(a*b)
    

In [431]:
summ = 0
n = 0
for t in tests:
    restorated_s = sen_restoration(norm_sen(t))
    good_s = good_sen(t)
    n += 1
    summ += score(good_s, restorated_s)
    if n > 10:
        break
print(summ / n)
    

['Na', 'jesieni', '1980', 'został', 'członkiem', 'zespołu', 'przygotowującego', 'do', 'druku', 'tygodnika', 'Solidarność', 'szefem', 'zespołu', 'był', 'Jerzy', 'Zieleński']
['Na', 'jesieni', '1980', 'został', 'członkiem', 'zespołu', 'przygotowującego', 'do', 'druku', 'Tygodnika', 'Solidarność', 'szefem', 'zespołu', 'był', 'Jerzy', 'Zieleński']
0.9971132578683505


In [433]:
print(unigrams['tygodnika'])
print(unigrams['solidarnosc'])

{'Tygodnika': 20, 'tygodnika': 129, 'TYGODNIKA': 2}
{'solidarność': 345, 'Solidarność': 724, 'SOLIDARNOŚĆ': 4, 'Solidarnosc': 4, 'solidarnosc': 1}


In [360]:
pd = "Chociaż znajdują się one na dwóch różnych krańcach świata, są niemal identyczne, bez własnego wizerunku, krajobrazu, atmosfery.".split()
#pd = "Mimo, iż takie miejsca tracą powoli dawną popularność, wielu współczesnym działaczom polskiej turystyki marzą się podobne widoki w Polsce i chcą przekształcić często niepowtarzalny krajobraz w betonowe ośrodki rozrywki, sportu i wypoczynku, do których za kilkadziesiąt lat nikt nie będzie chciał przyjeżdżać.".split()
pd ="Zabudowa krajobrazu prowadzi na ogół do urbanizacji terenów turystycznych, wskutek czego powstają wielkie miasta turystyczne, tzw. quasi-miasta, takie jak większość kurortów na Costa Brava w Hiszpanii, Acapulco lub Cancon w Meksyku.".split()

In [361]:
w = norm_sen(pd)
print(w)
#print(sen_restoration(w))
#print(score(good_sen(pd), sen_restoration(w)))

['zabudowa', 'krajobrazu', 'prowadzi', 'na', 'ogol', 'do', 'urbanizacji', 'terenow', 'turystycznych', 'wskutek', 'czego', 'powstaja', 'wielkie', 'miasta', 'turystyczne', 'tzw', 'quasimiasta', 'takie', 'jak', 'wiekszosc', 'kurortow', 'na', 'costa', 'brava', 'w', 'hiszpanii', 'acapulco', 'lub', 'cancon', 'w', 'meksyku']


In [241]:
print(bigrams['to']['okazja'])
print(bigrams['to']['okazją'])
print(bigrams['okazja']['do'])
print(bigrams['okazją']['do'])
print(unigrams['zielenski'])

649
18
2048
2967
{'Zieleński': 2, 'zieleński': 3}


In [242]:
print(bigrams['nad']['tą'])
print(bigrams['ta']['ustawa'])

8107
18667
